This is the first day of the HGSFP Graduate days lecture: Exploring the Milky Way: Statistical analysis of Gaia data

What are you going to learn overall: It's a very graphical data set. Its huge data amounts with pitfalls. We will get acquainted with the different data sets and use different statistical/machine learning algorithms to interpret them. We will learn/exploit domain knowledge and also try to give you the background information to the methods and to stimulate your problem solving abilities.

We will give you a little input and necessary information and then pose problems that you will adress in pairs and we will discuss in the group. We will work with jupyter notebooks using python to manipulate the data. The subsets of data are already here, but tools for acquiring more data and data vizualisation are provided and can be checked/extended. We will not manage to achieve everything, therefore motivated people can try and solve the missing bits at home.

Today we will start with the astrometric data set. Which mainly consists of phasespace information, i.e. parallaxes and proper motion. We will see the flaws of the data and look for overdensities in space and velocity space.

If you have questions / ideas please just throw them in.

Lecture Plan
Day 1: 
- Introduction to Milky Way and Stellar Science
- Introduction to Gaia Mission and DR2 Data
- Different data sets, historical context, what has been there until now
- (Overview of GDR2 science cases) Overview of what we plan to do
- How can we access this data, Short TAP tutorial introduce topcat
- First hands-on: Solar neighbourhood stars and a cluster
- What are parallaxes, how are they measured
- Give ADQL query to retrieve solar neighbourhood isolate Hyades
- Talk about spurious sources
- Show CMD explain stellar evolution (determine age of Hyades)
- Also show that in pm space
- Compare CMD with and without pm classification

Resources:
- [Gaia Overview](https://www.aanda.org/articles/aa/abs/2018/08/aa32727-18/aa32727-18.html)
- [Topcat](http://www.star.bris.ac.uk/%7Embt/topcat/)
- [GDR2mock](http://adsabs.harvard.edu/abs/2018PASP..130g4101R)
- [ADQL](https://www.gaia.ac.uk/data/gaia-data-release-1/adql-cookbook)
- [RUWE](https://www.cosmos.esa.int/web/gaia/dr2-known-issues)
- [PARSEC Isochrones](http://stev.oapd.inaf.it/cgi-bin/cmd)
- [GDR2_completeness](https://github.com/jan-rybizki/gdr2_completeness)

In [ ]:
# For reproducability we will write out the versions of the libraries 
%load_ext version_information
%version_information numpy, matplotlib, astropy, pyvo, scipy

In [ ]:
#load all the necessary libraries
%pylab inline
from astropy.io import fits

#a few utility functions are outsourced into other files in order to make the notebook concise
from plotting import plot_mollweide, plot_hrd
from utility import compute_ruwe

With the following ADQL query we get all Gaia DR2 stars with parallax > 20 mas. If you want to obtain the data set yourself you can have a look at the notebook 'Retrieving data day1' which was used to obtain the data for this notebook and essentially uses pyvo. You can also just enter the below ADQL query into topcat, once you are connected to the Vizier service (ivo://cds.vizier/tap): 

---

```SQL
SELECT *
FROM "I/345/gaia2"
WHERE parallax > 20```

---

Or at the browser interface of the [ESA Archive](https://gea.esac.esa.int/archive/):

---

```SQL
SELECT *
FROM gaiadr2.gaia_source
WHERE parallax > 20```

---

And we do the same with [GDR2mock](http://dc.g-vo.org/browse/gdr2mock/q) from GAVO (ivo://org.gavo.dc/tap):

---

```SQL
SELECT *
FROM gdr2mock.main
WHERE parallax > 20```

---

Input on:
- Structure of the query, possible use cases (join, upload, )
- Access the data (topcat, webinterface, pyvo)
- Data volume (Float 4 bytes * 1.6bn = 6.4GB)
- Data structure on the server (source_id, other indexed rows)
- GDR2mock (Galaxy model, extinction map, PARSEC Isochrones)

In [ ]:
# Loading the nearby sample
x = fits.getdata('../data/day_1/gdr2_near/nearby_sample_50pc.fits')
x = x.view(np.recarray)
print(len(x),x.dtype.names)
print('###########################################')
print('Now we load the GDR2 mock sample')
print('###########################################')
y = np.load('../data/day_1/gdr2mock_near/result_50pc.npy')
y = y.view(np.recarray)
print(len(y),y.dtype.names)

What is the expectation for this sample?
![MW](https://www.esa.int/var/esa/storage/images/esa_multimedia/images/2016/09/anatomy_of_the_milky_way/16112026-1-eng-GB/Anatomy_of_the_Milky_Way.jpg)

In [ ]:
# Expectations about the distribution of stars in the solar neighbourhood
plt.hist(np.divide(1,x.parallax)*1000, label = 'GDR2')
plt.hist(np.divide(1,y.parallax)*1000, label = 'GDR2mock')
plt.legend()
plt.xlabel("dist [pc]")
plt.ylabel("starcount")
plt.title('distance histogram solar neighbourhood')
plt.yscale("log")

In [ ]:
# Assessment of the problem with the data
plot_mollweide(x.l,x.b,np.divide(1,x.parallax)*1000,'dist [pc]',r'GDR2 $\varpi>20$')
plot_mollweide(x.l,x.b,x.phot_g_mean_mag,'G [mag]',r'GDR2 $\varpi>20$')
plot_mollweide(y.l,y.b,y.phot_g_mean_mag,'G [mag]',r'GDR2mock $\varpi>20$')

Discussion:
- Spurious sources

---

TASK (either in topcat or within this notebook): Try to identify good quality indicators, that help to get rid of the spurious sources.

Try for example:`parallax_error`, `astrometric_n_good_obs_al`, `astrometric_chi2_al`, `astrometric_excess_noise`, `visibility_periods_used` and `phot_bp_rp_excess_factor`.

In [ ]:
# This is what DPAC recommends (https://www.cosmos.esa.int/web/gaia/dr2-known-issues)
#UWE = UNIT WEIGHT ERROR = sqrt(astrometric_chi2_al/(astrometric_n_good_obs_al-5))
#And on top a dependence of that regarding to G mag and color.
ru = compute_ruwe(x)
# apply the cut and look at the result
astro_cut = (ru<1.4)
data = x[astro_cut]
plot_mollweide(data.l,data.b,np.divide(1,data.parallax)*1000,'dist [pc]',r'GDR2 $\varpi>20$, ruwe cut')

In [ ]:
# Photometric excess noise indicator, another quality indicator (Equation C2 from https://www.aanda.org/articles/aa/abs/2018/08/aa32727-18/aa32727-18.html)
# phot_bp_rp_excess_factor = (BP + RP flux) / G flux
photo_cut = (1 + 0.015 * (x.phot_bp_mean_mag - x.phot_rp_mean_mag)**2 < x.phot_bp_rp_excess_factor) & (1.3 + 0.06 * (x.phot_bp_mean_mag - x.phot_rp_mean_mag)**2 > x.phot_bp_rp_excess_factor)
data = x[photo_cut]
plot_mollweide(data.l,data.b,np.divide(1,data.parallax)*1000,'dist [pc]',r'GDR2 $\varpi>20$, photometric_excess_noise cut')

In [ ]:
data = x[photo_cut & astro_cut]
plot_mollweide(data.l,data.b,np.divide(1,data.parallax)*1000,'dist [pc]',r'GDR2 $\varpi>20$, both cuts')

In [ ]:
data = x[~(photo_cut & astro_cut)]
plot_mollweide(data.l,data.b,np.divide(1,data.parallax)*1000,'dist [pc]',r'GDR2 $\varpi>20$, what was cut')

In [ ]:
# Now in the distance histogram
data = x[photo_cut & astro_cut]
plt.hist(np.divide(1,data.parallax)*1000, label = 'GDR2',alpha = 0.5)
plt.hist(np.divide(1,y.parallax)*1000, label = 'GDR2mock', alpha = 0.5)
plt.legend()
plt.xlabel("dist [pc]")
plt.ylabel("starcount")
plt.title('distance histogram solar neighbourhood')
plt.yscale("log")
plt.show()
plt.clf()
plt.close()

In [ ]:
from plotting import plot_hrd
data = x[photo_cut & astro_cut]
plot_hrd(x,"all data",np.divide(1000,x.parallax),"distance [pc]")
plot_hrd(data,"good data",np.divide(1000,data.parallax),"distance [pc]")

Input:
- HRD
- Color magnitude diagram
- Stellar parameters

![HRD](http://sci.esa.int/science-e-media/img/26/ESA_Gaia_DR2_HRD_Gaia_625.jpg)

In [ ]:
data = x[~(photo_cut & astro_cut)]
plot_hrd(data,"cut out stars",np.divide(1000,data.parallax),"distance [pc]")

In [ ]:
hrd_cut = (x.parallax_over_error>10) & (x.phot_bp_mean_flux_over_error > 10) & (x.phot_rp_mean_flux_over_error > 10)
data = x[photo_cut & astro_cut & hrd_cut]
plot_hrd(data,"high SNR data",np.divide(1000,data.parallax),"distance [pc]")
plot_hrd(y,"GDR2mock",y.age,"age [Gyr]")
plot_hrd(y,"GDR2mock",y.feh,"metallicity [Fe/H]")

In [ ]:
data = x[photo_cut & astro_cut & hrd_cut]
plot_mollweide(data.ra,data.dec,np.divide(1,data.parallax)*1000,'dist [pc]',r'GDR2 $\varpi>20$, both cuts')
file = '../data/day_1/topcat_hrd/cleaned_data.fits'
import os
try:
    os.remove(file)
except OSError:
    pass
fits.writeto(file,data)


Data Exploration: Load that dataset in topcat and try to isolate the cluster

Plot and cut in: 
- Position on the sky (glon vs glat or ra vs dec)
- Distance (1/parallax * 1000)
- Proper motion (pmra vs pmdec)

Just inspect
- HRD (BP-RP vs G + 5 * log10(parallax/100))
- Teff
- Radial velocity

A historical note on the CMD? CMDs of globular cluster. Stars of same distance and same age...

In [ ]:
# We load the isolated cluster
h = fits.getdata("../data/day_1/topcat_hrd/Hyades_only.fits")
h = h.view(np.recarray)

color_h = h.phot_bp_mean_mag - h.phot_rp_mean_mag
abs_mag_h = h.phot_g_mean_mag + 5 * np.log10(np.divide(h.parallax,100))
plt.plot(color_h,abs_mag_h,',')
plt.gca().invert_yaxis()
plt.ylabel("G + 5 log10(parallax/100)")
plt.xlabel("BP-RP")
plt.title("CMD Hyades")

In [ ]:
# We load the PARSEC Isochrones (stellar evolutionary tracks)
p = np.load('../data/day_1/parsec_isochrones/no_ext_grid.npy')
p = p.view(np.recarray)
print(len(p),p.dtype.names)

In [ ]:
color = p.gaia_bp - p.gaia_rp
abs_mag = p.gaia_g
plt.plot(color,abs_mag,',',label = "PARSEC")
plt.plot(color_h,abs_mag_h,',',label = "Hyades")
plt.gca().invert_yaxis()
plt.ylabel("G + 5 log10(parallax/100)")
plt.xlabel("BP-RP")
plt.title("CMD")
plt.legend()

In [ ]:
# Slicing the model in FEH space

feh_values = np.unique(p.feh)
for feh in feh_values:
    cut = np.where(p.feh == feh)
    plt.plot(color[cut],abs_mag[cut],',',label = "PARSEC Feh = %.2f" %(feh))
    color_h = h.phot_bp_mean_mag - h.phot_rp_mean_mag
    abs_mag_h = h.phot_g_mean_mag + 5 * np.log10(np.divide(h.parallax,100))
    plt.plot(color_h,abs_mag_h,',',label = "Hyades")
    plt.gca().invert_yaxis()
    plt.ylabel("G + 5 log10(parallax/100)")
    plt.xlabel("BP-RP")
    plt.title("CMD")
    plt.legend()
    plt.show()
    plt.clf()
    plt.close()

The metallicity of the Hyades is supersolar, approximately 0.25 dex

In [ ]:
# Slicing the model in age for a specific metallicity
age_values = np.unique(p.age)
for i,age in enumerate(age_values):
    print(i)
    cut = np.where(np.logical_and(p.feh == 0.25,p.age == age))
    #im = plt.scatter(color[cut],abs_mag[cut],s=5,c=p.age[cut], norm = LogNorm(),label = "PARSEC Feh = %.2f" %(feh))
    plt.plot(color[cut],abs_mag[cut],',',label = "PARSEC Feh = 0.25 & Age = %.3f Myr" %(age*1000))
    color_h = h.phot_bp_mean_mag - h.phot_rp_mean_mag
    abs_mag_h = h.phot_g_mean_mag + 5 * np.log10(np.divide(h.parallax,100))
    plt.plot(color_h,abs_mag_h,',',label = "Hyades")
    plt.gca().invert_yaxis()
    #plt.colorbar(im, label = "age")
    plt.ylabel("G + 5 log10(parallax/100)")
    plt.xlabel("BP-RP")
    plt.title("CMD")
    plt.legend()
    plt.show()
    plt.clf()
    plt.close()

The Age of the Hyades is between 500 and 700 Myr.

In [ ]:
# Plotting a good model representation and look at the mass

cut = np.where(np.logical_and(p.feh == 0.25,p.age == age_values[43]))
im = plt.scatter(color[cut],abs_mag[cut],s=5,c=p.mass[cut], label = "PARSEC Hyades Isochrone" )
color_h = h.phot_bp_mean_mag - h.phot_rp_mean_mag
abs_mag_h = h.phot_g_mean_mag + 5 * np.log10(np.divide(h.parallax,100))
plt.plot(color_h,abs_mag_h,',',label = "Hyades")
plt.gca().invert_yaxis()
plt.colorbar(im, label = "mass")
plt.ylabel("G + 5 log10(parallax/100)")
plt.xlabel("BP-RP")
plt.title("CMD Mass sequence of an Isochrone")
plt.legend()
plt.show()
plt.clf()
plt.close()

What have we seen: CMD powerful tool to constrain teff, age, mass
On Thursday we will learn how to infer those quantities in a Bayesian framework from field stars.

Tomorrow Maria will talk about time series analysis.

We will meet again on Wednesday and then we will look at close stellar encounters.